<a href="https://colab.research.google.com/github/RingoKid/EuroSAT-DeepLearning/blob/main/EuroSATxreBEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the required libraries

In [ ]:
import os
import subprocess
import sys

# Suppress outputs for pip install
subprocess.run(["pip", "install", "configilm"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["pip", "install", "lightning"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["pip", "install", "lmdb"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Suppress output for git clone
subprocess.run(["git", "clone", "https://git.tu-berlin.de/rsim/reben-training-scripts.git"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Change directory without output
# os.chdir('/content/reben-training-scripts')
# from reben_publication.BigEarthNetv2_0_ImageClassifier import BigEarthNetv2_0_ImageClassifier
sys.path.append('/content/reben-training-scripts')


In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow-cpu

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.image import resize

import json
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import torch.optim as optim
from torch.nn import BCEWithLogitsLoss


from reben_publication.BigEarthNetv2_0_ImageClassifier import BigEarthNetv2_0_ImageClassifier
from configilm.extra.BENv2_utils import band_combi_to_mean_std, STANDARD_BANDS




# Load the EuroSAT dataset (all bands version)
The EuroSAT dataset is loaded using `tfds.load()`. We use the `"eurosat/all"` version to get all spectral bands. The `with_info=True` flag provides metadata about the dataset (`info`), which gives us additional context like the number of classes, features, etc.


In [ ]:
# Save the dataset to a specific directory
save_path = '/content/eurosat_dataset'
dataset, info = tfds.load("eurosat/all", split='train', with_info=True, data_dir=save_path)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27000 [00:00<?, ? examples/s]

Shuffling /content/eurosat_dataset/eurosat/all/incomplete.I4OUEN_2.0.0/eurosat-train.tfrecord*...:   0%|      …

Dataset eurosat downloaded and prepared to /content/eurosat_dataset/eurosat/all/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# Take one sample from the dataset
for sample in dataset.take(1):
    # Access the Sentinel-2 image and label
    image = sample['sentinel2']  # Shape: (64, 64, 13)
    label = sample['label']  # Corresponding label

    # Print the shapes
    print(f"Image Shape: {image.shape}")
    print(f"Label: {label.numpy()}")


Image Shape: (64, 64, 13)
Label: 8


In [ ]:
dataset

<_PrefetchDataset element_spec={'filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sentinel2': TensorSpec(shape=(64, 64, 13), dtype=tf.float32, name=None)}>

### Load the BigEarthNet

### Check the input size

In [ ]:
# Access the input configuration
input_size = model.config.image_size  # BigEarthNet configuration typically stores the input size

print(f"Expected input image size: {input_size} x {input_size}")

NameError: name 'model' is not defined

In [ ]:
# Inspect the first layer of the model
first_layer = model.model.vision_encoder.conv1  # Adjust path based on the model structure
print(first_layer)

# For PyTorch, the kernel size or input channels may hint at the expected input size.


### Filter the dataset to drop 3 bands

In [ ]:
def filter_eurosat_bands_tf(sentinel2_tensor):
    """
    Filters the Sentinel-2 tensor to retain only the 10 relevant bands used in BigEarthNet.

    Args:
        sentinel2_tensor: TensorFlow tensor of shape (H, W, 13) containing Sentinel-2 bands.

    Returns:
        A TensorFlow tensor of shape (H, W, 10) with the filtered bands.
    """
    # Define indices for the 10 relevant bands
    indices_to_keep = [1, 2, 3, 4, 5, 6, 7, 8, 10, 11]  # Indices for B02, B03, ..., B12

    # Use tf.gather to filter bands along the last axis
    filtered_tensor = tf.gather(sentinel2_tensor, indices_to_keep, axis=-1)

    return filtered_tensor


In [ ]:
# A function to apply the filtering to each sample in the dataset
def preprocess_image(sentinel2_tensor):
    # Convert the TensorFlow tensor to a numpy array
    sentinel2_tensor_np = sentinel2_tensor.numpy()

    # Apply the filter function
    filtered_tensor = filter_eurosat_bands(sentinel2_tensor_np)

    # Convert the numpy array back to a TensorFlow tensor
    filtered_tensor_tf = tf.convert_to_tensor(filtered_tensor, dtype=tf.float32)

    return filtered_tensor_tf


In [ ]:
# Apply the preprocessing to the dataset using tf.py_function
def tf_preprocess_image(features):
    # Extract the relevant fields from the dictionary
    sentinel2_tensor = features['sentinel2']
    label = features['label']

    # Filter the bands using native TensorFlow operations
    filtered_image = filter_eurosat_bands_tf(sentinel2_tensor)

    return {'filename': features['filename'], 'sentinel2': filtered_image, 'label': label}


In [ ]:
# Apply the preprocessing to the dataset
filtered_dataset = dataset.map(tf_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
print(filtered_dataset)

<_ParallelMapDataset element_spec={'filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'sentinel2': TensorSpec(shape=(64, 64, 10), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


### Resize the dataset to 120x120

In [ ]:
# Define a function to resize the image while keeping the filename intact
def resize_sample_with_filename(sample):
    """
    Resize the Sentinel-2 image in the dataset to 120x120 for all bands and keep the filename intact.
    Args:
        sample: A dictionary containing 'filename', 'sentinel2', and 'label'.
    Returns:
        A dictionary with resized 'sentinel2', the same 'label', and 'filename'.
    """
    image = sample['sentinel2']  # Extract the Sentinel-2 image
    label = sample['label']      # Extract the label
    filename = sample['filename']  # Keep the filename

    # Resize the image to 120x120 for all bands
    resized_image = tf.image.resize(image, [120, 120])

    return {'filename': filename, 'sentinel2': resized_image, 'label': label}

# Apply the resize function to the entire dataset
resized_dataset = filtered_dataset.map(resize_sample_with_filename, num_parallel_calls=tf.data.AUTOTUNE)
print(resized_dataset)

<_ParallelMapDataset element_spec={'filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'sentinel2': TensorSpec(shape=(120, 120, 10), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


## Normalization

In [ ]:
from configilm.extra.BENv2_utils import band_combi_to_mean_std, STANDARD_BANDS

# Define mean and std for the 10 Sentinel-2 bands (STANDARD_BANDS[10])
means, stds = band_combi_to_mean_std(STANDARD_BANDS[10], interpolation="120_bilinear")
print(means)
print(stds)

[ 438.37207031  614.05566406  588.40960693  942.74768066 1769.84863281
 2049.47583008 2193.29199219 2235.48681641 1568.21154785  997.71508789]
[ 607.02685547  603.29681396  684.56884766  727.57843018 1087.42883301
 1261.43029785 1369.3717041  1342.49047852 1063.9197998   806.88464355]


In [ ]:
# Define the normalization function
def normalize_image(features):
    """
    Normalize the Sentinel-2 image tensor using z-score normalization.

    Args:
        features: Dictionary containing the 'sentinel2' tensor and other fields (e.g., 'label').

    Returns:
        Dictionary with normalized image and other fields unchanged.
    """
    # Extract image tensor and label
    sentinel2_tensor = features['sentinel2']
    label = features['label']
    filename = features['filename']

    # Perform z-score normalization
    normalized_image = (sentinel2_tensor - tf.constant(means, shape=(1, 1, 10), dtype=tf.float32)) / \
                       tf.constant(stds, shape=(1, 1, 10), dtype=tf.float32)

    # Return the normalized image with label and filename unchanged
    return {'filename': filename, 'sentinel2': normalized_image, 'label': label}

# Apply the normalization function to the dataset
normalized_dataset = resized_dataset.map(normalize_image, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
# Check the dataset structure
for sample in normalized_dataset.take(1):
    print("Filename:", sample['filename'].numpy())  # Filename remains intact
    print("Image Shape:", sample['sentinel2'].shape)  # Should still be (120, 120, 10)
    print("Label:", sample['label'].numpy())  # Label remains unchanged

Filename: b'River_15.tif'
Image Shape: (120, 120, 10)
Label: 8


In [ ]:

# Function to compute mean and std per band for normalized dataset
def validate_normalized_dataset(dataset):
    # Initialize accumulators for mean and std computation
    total_sum = tf.zeros((10,), dtype=tf.float32)
    total_sum_sq = tf.zeros((10,), dtype=tf.float32)
    total_count = 0

    # Iterate through the dataset
    for sample in dataset:
        image = sample['sentinel2']  # Access the normalized image tensor
        # Flatten the bands (preserves the last axis)
        reshaped_image = tf.reshape(image, [-1, 10])

        # Sum up the values
        total_sum += tf.reduce_sum(reshaped_image, axis=0)
        total_sum_sq += tf.reduce_sum(reshaped_image**2, axis=0)
        total_count += tf.shape(reshaped_image)[0]

    # Calculate mean and std
    mean = total_sum / tf.cast(total_count, tf.float32)
    std = tf.sqrt((total_sum_sq / tf.cast(total_count, tf.float32)) - mean**2)

    return mean.numpy(), std.numpy()

# Validate the normalized dataset
mean_per_band, std_per_band = validate_normalized_dataset(normalized_dataset)

# Print results
for i, (mean, std) in enumerate(zip(mean_per_band, std_per_band)):
    print(f"Band {i + 1}: Mean={mean:.4f}, Std={std:.4f}")


Band 1: Mean=1.1183, Std=0.5395
Band 2: Mean=0.7092, Std=0.6445
Band 3: Mean=0.5232, Std=0.8578
Band 4: Mean=0.3525, Std=0.7778
Band 5: Mean=0.2144, Std=0.7905
Band 6: Mean=0.2573, Std=0.8603
Band 7: Mean=0.0788, Std=0.8094
Band 8: Mean=-1.1198, Std=0.3008
Band 9: Mean=0.2373, Std=0.9413
Band 10: Mean=0.1493, Std=0.9412


### Split the data to train, test and validate

#### Convert Entire Dataset

In [ ]:
import torch.nn.functional as F

class TensorflowToTorchDataset(Dataset):
    def __init__(self, tf_dataset):
        # Convert TensorFlow dataset to a list of samples
        self.data = list(tf_dataset.as_numpy_iterator())

    def __len__(self):
        # Return the total number of samples
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        image = torch.tensor(sample['sentinel2'], dtype=torch.float32).permute(2, 0, 1)  # [channels, height, width]
        label = F.one_hot(torch.tensor(sample['label']), num_classes=10).float()  # One-hot encode
        return image, label


### Split the Dataset

In [ ]:
# Convert TensorFlow dataset to PyTorch dataset
torch_dataset = TensorflowToTorchDataset(normalized_dataset)

dataset_size = len(torch_dataset)


train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size  # Ensure no leftover samples

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(torch_dataset, [train_size, val_size, test_size])

In [ ]:
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)

print(f"Training dataset size: {train_size}")
print(f"Validation dataset size: {val_size}")
print(f"Test dataset size: {test_size}")


Training dataset size: 21600
Validation dataset size: 2700
Test dataset size: 2700


In [ ]:
# Define batch size
batch_size = 64

# Create DataLoaders for train, validation, and test datasets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example: Check a single batch from train_loader
for images, labels in train_loader:
    print(f"Batch Images Shape: {images.shape}")
    print(f"Batch Labels Shape: {labels.shape}")
    break  # Check only the first batch

Batch Images Shape: torch.Size([64, 10, 120, 120])
Batch Labels Shape: torch.Size([64, 10])


### Load the model

In [ ]:
model = BigEarthNetv2_0_ImageClassifier.from_pretrained(
  "BIFOLD-BigEarthNetv2-0/resnet101-s2-v0.1.1"
)
print(model)

/usr/local/lib/python3.10/dist-packages/configilm/ConfigILM.py:134: UserWarning: Keyword 'img_size' unknown. Trying to ignore and restart creation.
  warnings.warn(f"Keyword '{failed_kw}' unknown. Trying to ignore and restart creation.")


BigEarthNetv2_0_ImageClassifier(
  (model): ConfigILM(
    (vision_encoder): ResNet(
      (conv1): Conv2d(10, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act2): ReLU(inplace=True)
          (aa): Identity()
          (conv3): Conv2d(64, 256, kernel_s

In [ ]:
print(model.model)  # Check the structure of the ConfigILM model


ConfigILM(
  (vision_encoder): ResNet(
    (conv1): Conv2d(10, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1

In [ ]:
# Define the number of output classes for EuroSAT
num_classes = 10

# Replace the final fully connected layer
model.model.vision_encoder.fc = nn.Linear(
    model.model.vision_encoder.fc.in_features, num_classes
)

# Verify the change
print(model.model.vision_encoder.fc)

Linear(in_features=2048, out_features=10, bias=True)


### Freezing Layers


In [ ]:
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the final layer
for param in model.model.vision_encoder.fc.parameters():
    param.requires_grad = True


Set up your optimizer to only optimize the unfreezed parameters

In [ ]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()


In [ ]:
# Hyperparameters
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()  # Set the model to training mode
    running_loss = 0.0  # Track loss across batches

    for images, labels in train_loader:
        # Move data to the same device as the model
        images = images.to(device)
        labels = labels.to(device)

        # Zero out gradients from the last iteration
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute the loss
        loss = criterion(outputs, labels.float())

        # Backward pass and weight update
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        running_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = running_loss / len(train_loader)
    print(f"Training Loss: {avg_loss:.4f}")

    # Validation step after each epoch
    model.eval()  # Switch to evaluation mode
    val_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # No gradient calculation during evaluation
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels.float())
            val_loss += loss.item()

            # Convert outputs to binary predictions (for binary/multilabel classification)
            preds = (torch.sigmoid(outputs) > 0.5).int()
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct_predictions / total_predictions
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

print("Training complete!")


Epoch 1/5
Training Loss: 0.1556
Validation Loss: 0.1077, Accuracy: 0.9617
Epoch 2/5
Training Loss: 0.1131
Validation Loss: 0.0924, Accuracy: 0.9675
Epoch 3/5
Training Loss: 0.1030
Validation Loss: 0.0834, Accuracy: 0.9699
Epoch 4/5
Training Loss: 0.0973
Validation Loss: 0.0792, Accuracy: 0.9724
Epoch 5/5
Training Loss: 0.0931
Validation Loss: 0.0760, Accuracy: 0.9734
Training complete!
